# Homework 1: Preprocessing and Text Classification

Student Name: Na Chang

Student ID: 858604

Python version used: 2.7

## General info

<b>Due date</b>: 11pm, Sunday March 18th

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be using a corpus of tweets to do tokenisation of hashtags and build polarity classifers using bag of word (BOW) features.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a few minutes, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Preprocessing

<b>Instructions</b>: For this homework we will be using the tweets in the <i>twitter_samples</i> corpus included with NLTK. You should start by accessing these tweets. Use the <i>strings</i> method included in the NLTK corpus reader for <i>twitter_samples</i> to access the tweets (as raw strings). Iterate over the full corpus, and print out the average length, in characters, of the tweets in the corpus. (0.5)


In [1]:
import nltk

nltk.download("twitter_samples")

from nltk.corpus import twitter_samples

tweets = twitter_samples.strings()
avgLength = int(sum(map(len, tweets))/float(len(tweets)))
print "Average Length in Char: ", int(avgLength)

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/Rena/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
Average Length in Char:  103


<b>Instructions</b>: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. You should use a regular expression to extract all hashtags of length 8 or longer which consist only of lower case letters (other than the # at the beginning, of course, though this should be stripped off as part of the extraction process). Do <b>not</b> tokenise the entire tweet as part of this process. The hashtag might occur at the beginning or the end of the tweet; you should double-check that you aren't missing any. After you have collected them into a list, print out number of hashtags you have collected: for full credit, you must get the exact number that we expect.  (1.0)

In [2]:
import re

hashtags = []

for tweet in tweets:
	single_tags = re.findall(
        r'(?<=\s#)[a-z]{8,}\s|(?<=^#)[a-z]{8,}|(?<=\s#)[a-z]{8,}$', tweet)
	if single_tags:
		for tag in single_tags:
			hashtags.append(tag.strip())
print "Total number of Hashtags: ",len(hashtags)

Total number of Hashtags:  1411


<b>Instructions</b>: Now, tokenise the hashtags you've collected. To do this, you should implement a reversed version of the MaxMatch algorithm discussed in class (and in the reading), where matching begins at the end of the hashtag and progresses backwards. NLTK has a list of words that you can use for matching, see starter code below. Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatiser before matching. Note that the list of words is incomplete, and, if you are unable to make any longer match, your code should default to matching a single letter. Create a new list of tokenised hashtags (this should be a list of lists of strings) and use slicing to print out the last 20 hashtags in the list. (1.0)

In [3]:
nltk.download('words')
words = set(nltk.corpus.words.words())

nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()


def lemmatize(word):
	###return the lemma form of input token###
	###first lemmatize as verb###
	###if does not change, lemmatize as noun#
	lemma = lemmatizer.lemmatize(word,'v')
	if lemma == word:
		lemma = lemmatizer.lemmatize(word,'n')
	return lemma


def maxMatchReverse(tag, dictionary, splitTokens):
	###input a string, a linguistic dictionary, and a list for splitted tokens###
	###while loop try to match the tail of string###
	###recursion takes the remaining string from the head###
	if not tag:
		return splitTokens
	length = len(tag)
	startPos = 0
	while startPos != length - 1:
		lastPart = tag[startPos:]
		remain = tag[:startPos]
		if lastPart in dictionary:
			splitTokens = [lastPart] + splitTokens
			return maxMatchReverse(remain, dictionary, splitTokens)
		elif lemmatize(lastPart) in dictionary:
			splitTokens = [lastPart] + splitTokens
			return maxMatchReverse(remain, dictionary, splitTokens)
		startPos += 1
	splitTokens = [tag[startPos:]] + splitTokens
	return maxMatchReverse(tag[:startPos], dictionary, splitTokens)


def maxMatch(tag, dictionary, splitTokens):
    ###the reversed version of function above
	if not tag:
		return splitTokens
	length = len(tag)
	while length != 1:
		firstPart = tag[:length]
		remain = tag[length:]
		if firstPart in dictionary:
			splitTokens.append(firstPart)
			return maxMatch(remain, dictionary, splitTokens)
		elif lemmatize(firstPart) in dictionary:
			splitTokens.append(firstPart)
			return maxMatch(remain, dictionary, splitTokens)
		length -= 1
	splitTokens.append(tag[:length])
	return maxMatch(tag[length:], dictionary, splitTokens)

tokenisedTagsRev = []
tokenisedTagsSta = []
for tag in hashtags:
	tokenisedTagsRev.append(maxMatchReverse(tag, words, []))
	tokenisedTagsSta.append(maxMatch(tag, words, []))
print "Last 20 Tokenised \n",tokenisedTagsRev[(len(tokenisedTagsRev)-20):]

[nltk_data] Downloading package words to /Users/Rena/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Rena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Last 20 Tokenised 
[[u'leaders', u'debate'], [u'wow', u'campaign'], [u'social', u'security'], [u'tory', u'lies'], [u'election'], [u'biased', u'b', u'b', u'c'], [u'labour', u'doorstep'], [u'biased', u'b', u'b', u'c'], [u'li', u'blab', u'con'], [u'b', u'b', u'c', u'debate'], [u'mi', u'li', u'fandom'], [u'u', u'k', u'parliament'], [u'bedroom', u'tax'], [u'disability'], [u'canna', u'bis'], [u'vote', u'green'], [u'l', u'lan', u'el', u'li', u'h', u'u', u'stings'], [u'bedroom', u'tax'], [u'disability'], [u'bankrupt']]


### Extra Credit (Optional)
<b>Instructions</b>: Implement the forward version of the MaxMatch algorithm as well, and print out all the hashtags which give different results for the two versions of MaxMatch. Your main task is to come up with a good way to select which of the two segmentations is better for any given case, and demonstrate that it works significantly better than using a single version of the algorithm for all hashtags. (1.0)

In [4]:
###Rational: the correctly split tokens have 2 characteristics:
###1.have very little single character tokens
###2.its original form exist in the dictionary
###countChar penalise single char token
###countOrigin reward the token that exist in the words as its original form
def countChar(tokenList):
	count = 0
	if tokenList:
		for token in tokenList:
			if len(token) == 1:
				count -= 1
	return count

def countOrigin(tokenList):
	count = 0
	if tokenList:
		for token in tokenList:
			if token in words:
				count += 1
	return count

def matchPick(maxResult, maxRevResult):
	maxAcc = 0
	maxRevAcc = 0
	maxAcc = countChar(maxResult) + countOrigin(maxResult)
	maxRevAcc = countChar(maxRevResult) + countOrigin(maxRevResult)
	if maxAcc > maxRevAcc:
		return maxMatch 
	else:
		return maxMatchReverse
    
    
tagLength = len(hashtags)
difTagSta = []
difTagRev = []
difTag = {}
counter = 0
while counter < tagLength: 
	sta = tokenisedTagsSta[counter]
	rev = tokenisedTagsRev[counter]
	if len(sta) != len(rev):
		difTagSta.append(sta)
		difTagRev.append(rev)
		difTag["".join(sta)] = (sta,rev)
	else:
		innerCounter = 0
		while innerCounter < len(sta):
			if sta[innerCounter] != rev[innerCounter]:
				difTagSta.append(sta)
				difTagRev.append(rev)
				difTag["".join(sta)] = (sta, rev)
				break
			innerCounter += 1
	counter += 1
###for validation purpose, print out the collection of different tag
###print "tag", difTag

screenedTag = []
for tag in difTag.keys():
	(maxResult, maxRevResult) = difTag[tag]
	func = matchPick(maxResult, maxRevResult)
	screenedTag.append(func(tag, words, []))

###for validation purpose, print out the screened tag
###print "screened Tag \n", screenedTag

###corrected example
###u'supersmash', u'votingsystemturnoff', u'gamedesign' etc

###print "done"

screened Tag 
[[u'no', u'mo', u'relies'], [u'v', u'o', u't', u'eu', u'kip'], [u'rud', u'ram', u'ade', u'v', u'i'], [u'the', u'reis', u'no', u'escape'], [u'super', u'smash'], [u'messenger', u'for', u'aday'], [u'ask', u'fa', u'rage'], [u'doc', u'open', u'hag', u'en'], [u'al', u'exs', u'almond'], [u's', u'tal', u'ber', u't'], [u'manche', u'st', u'er'], [u'voting', u'system', u'turnoff'], [u'de', u'f', u'f', u'o', u'not', u'p', u'c'], [u'star', u'tups'], [u'pro', u'sec', u'c', u'o'], [u'p', u'e', u'na', u'c', u'ova'], [u'st', u'ur', u'wars'], [u'g', u'i', u'ach', u'ie', u'tit', u'ti', u'wedding'], [u'f', u'rid', u'ay', u'feeling'], [u'hyp', u'no', u'toad'], [u'z', u'or', u'r', u'ore', u'tur', u'ms'], [u'cloud', u'coo', u'ko', u'o'], [u'mi', u'li', u'brand'], [u'li', u'v', u'eon', u'st', u'rea', u'mate'], [u'liar', u'liar', u'pant', u'son', u'fire'], [u'use', u'your', u'sense'], [u'takes', u'one', u'to', u'k', u'no', u'wone'], [u'add', u'me', u'ons', u'nap', u'chat'], [u'c', u'h', u'ama', u

## Text classification (Not Optional)

<b>Instructions</b>: The twitter_sample corpus has two subcorpora corresponding to positive and negative tweets. You can access already tokenised versions using the <i> tokenized </i> method, as given in the code sample below. Iterate through these two corpora and build training, development, and test sets for use with Scikit-learn. You should exclude stopwords (from the built-in NLTK list) and tokens with non-alphabetic characters (this is very important you do this because emoticons were used to build the corpus, if you don't remove them performance will be artificially high). You should randomly split each subcorpus, using 80% of the tweets for training, 10% for development, and 10% for testing; make sure you do this <b>before</b> combining the tweets from the positive/negative subcorpora, so that the sets are <i>stratified</i>, i.e. the exact ratio of positive and negative tweets is preserved across the three sets. (1.0)

In [5]:
positive_tweets = nltk.corpus.twitter_samples.tokenized("positive_tweets.json")
negative_tweets = nltk.corpus.twitter_samples.tokenized("negative_tweets.json")

import re
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

def removeStopwordsNPattern(tweetList, stopwords):
    ###remove stop words, and words that are not alphabetical
    ###from the result generated, remove empty list
	newTweetList = []
	if tweetList:
		for tweet in tweetList:
			newTweet = []
			for word in tweet:
				if word.isalpha():
					if word not in stopwords:
						newTweet.append(word)
			newTweetList.append(newTweet)
	cleanedTweetList = []
	for tweets in newTweetList:
		if tweets:
			cleanedTweetList.append(tweets)
	return cleanedTweetList

cleanedPosTweet = removeStopwordsNPattern(positive_tweets, stopwords)
cleanedNegTweet = removeStopwordsNPattern(negative_tweets, stopwords)



def getBOW(text):
    ###sourced from the example from lecture-classifier
	BOW = {}
	for word in text:
		BOW[word] = BOW.get(word, 0) + 1
	return BOW


def prepareTweetData(vectorizer, posTweet, negTweet, 
                     featureExtractor, train = True):
    ###generate matrix-classification dataset
	featureMatrix = []
	classification = []
	if posTweet:
		for tweet in posTweet:
			featureDict = featureExtractor(tweet)
			featureMatrix.append(featureDict)
			classification.append("POS")
	if negTweet:
		for tweet in negTweet:
			featureDict = featureExtractor(tweet)
			featureMatrix.append(featureDict)
			classification.append("NEG")
	if train:
		dataset = vectorizer.fit_transform(featureMatrix)
	else:
		dataset = vectorizer.transform(featureMatrix)
	return dataset, classification

from sklearn.model_selection import train_test_split

posTrain, posValid = train_test_split(cleanedPosTweet, test_size=0.2)
posDev, posTest = train_test_split(posValid, test_size=0.5)

negTrain, negValid = train_test_split(cleanedNegTweet, test_size=0.2)
negDev, negTest = train_test_split(negValid, test_size=0.5)

from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer()

featureTrain, classTrain = prepareTweetData(
    vectorizer, posTrain, negTrain, getBOW)


[nltk_data] Downloading package stopwords to /Users/Rena/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<b>Instructions</b>: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation (hyper)parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do <b>not</b> use crossvalidation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the accuracy with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output. (1.0)

In [6]:
###The alpha and C value is picked via online reference and validated via GridSearchCV
###To get a stable performance and avoid overfitting
###The GridSearchCV is commented out to avoid crossvalidation for the training set in the programme

'''from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

alpha_collection = {}
alpha_collection['alpha'] = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
c_collection = {}
c_collection['C'] = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

nb_clf = MultinomialNB()
lg_clf = LogisticRegression(random_state = 333)

grid_nbclf = GridSearchCV(nb_clf, alpha_collection, 'accuracy')
grid_nbclf.fit(featureTrain, classTrain)
print "NB Optimal",grid_nbclf.best_params_

grid_lgclf = GridSearchCV(lg_clf, c_collection, 'accuracy')
grid_lgclf.fit(featureTrain, classTrain)
print "LG Optimal", grid_lgclf.best_params_'''

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

clf = MultinomialNB(alpha=0.9)
clf.fit(featureTrain, classTrain)

clfSmall = MultinomialNB(alpha = 0.1)
clfSmall.fit(featureTrain, classTrain)
clfMedium = MultinomialNB(alpha = 0.5)
clfMedium.fit(featureTrain, classTrain)

logreg = LogisticRegression(C=0.6)
logreg.fit(featureTrain, classTrain)

logregSmall = LogisticRegression(C = 0.1)
logregSmall.fit(featureTrain, classTrain)
logregLarge = LogisticRegression(C = 0.9)
logregLarge.fit(featureTrain, classTrain)


from sklearn.metrics import accuracy_score, classification_report
def checkResults(predictions, classifications):
	print "accuracy"
	print accuracy_score(classifications, predictions)
	print classification_report(classifications, predictions)

featureDev, classDev = prepareTweetData(
    vectorizer, posDev, negDev, getBOW, train=False)

devPredNB = clf.predict(featureDev)
devPredNBSmall = clfSmall.predict(featureDev)
devPredNBMedium = clfMedium.predict(featureDev)

devPredLR = logreg.predict(featureDev)
devPredLRSmall = logregSmall.predict(featureDev)
devPredLRLarge = logregLarge.predict(featureDev)


print "Development Data Result"
print "Naive Bayes Result - Optimal"
checkResults(devPredNB, classDev)
print "Naive Bayes Result - Small"
checkResults(devPredNBSmall, classDev)
print "Naive Bayes Result - Medium"
checkResults(devPredNBMedium, classDev)
print "Logistic Regression Result - Optimal"
checkResults(devPredLR, classDev)
print "Logistic Regression Result - Small"
checkResults(devPredLRSmall, classDev)
print "Logistic Regression Result - Large"
checkResults(devPredLRLarge, classDev)

Development Data Result
Naive Bayes Result - Optimal
accuracy
0.710261569416
             precision    recall  f1-score   support

        NEG       0.70      0.73      0.72       496
        POS       0.72      0.69      0.70       498

avg / total       0.71      0.71      0.71       994

Naive Bayes Result - Small
accuracy
0.69416498994
             precision    recall  f1-score   support

        NEG       0.69      0.71      0.70       496
        POS       0.70      0.68      0.69       498

avg / total       0.69      0.69      0.69       994

Naive Bayes Result - Medium
accuracy
0.70523138833
             precision    recall  f1-score   support

        NEG       0.70      0.72      0.71       496
        POS       0.71      0.69      0.70       498

avg / total       0.71      0.71      0.71       994

Logistic Regression Result - Optimal
accuracy
0.72032193159
             precision    recall  f1-score   support

        NEG       0.70      0.77      0.73       496
        PO

<b>Instructions</b>: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both accuracy and macroaveraged f-score for each classifier. Be sure to label your output. (0.5)

In [7]:
featureTest, classTest = prepareTweetData(vectorizer, posTest, negTest, getBOW, train=False)
testPredNB = clf.predict(featureTest)
testPredLG = logreg.predict(featureTest)

print "Test Data Result"
print "Naive Bayes Result"
checkResults(testPredNB, classTest)
print "Logistic Regression Result"
checkResults(testPredLG, classTest)

Test Data Result
Naive Bayes Result
accuracy
0.73138832998
             precision    recall  f1-score   support

        NEG       0.73      0.74      0.73       496
        POS       0.74      0.72      0.73       498

avg / total       0.73      0.73      0.73       994

Logistic Regression Result
accuracy
0.760563380282
             precision    recall  f1-score   support

        NEG       0.74      0.80      0.77       496
        POS       0.78      0.72      0.75       498

avg / total       0.76      0.76      0.76       994

